## Loading package dependencies

`%pip install` commands need only be run once per JupyterHub session.
If you restart your JupyterHub server, they should be re-installed.

If you want to ensure you are running latest version of packages, use `pip install --upgrade ...`

Notebook dependencies may be pre-installed on custom notebook images in future iterations.

The cell below uses `%%capture` to capture the (usually long) output of pip installs in a variable,
so it does not fill your console with large output cells

In [1]:
%%capture pipoutput
%pip install trino[sqlalchemy] python-dotenv pandas
%pip install osc-ingest-tools

## Load credentials
OS-Climate convention is to store credentials using the `dotenv` file `credentials.env`

This cell loads the environment variables defined in `credentials.env` and sets them in python's
standard `os.environ` environment variable dictionary.

You can obtain your own `credentials.env` file as part of the OS-climate onboarding process.
You can upload this file from your computer using the 'upload' button on the left of the Jupyter console.

Here we are using a standard OSC utility `load_credentials_dotenv()` that loads `credentials.env`.
It expects to find this file at the top-level directory in the file browsing menu on the left hand side
of the Jupyter console.

In [2]:
import osc_ingest_trino as osc
osc.load_credentials_dotenv()

## Connect to trino with sqlalchemy engine

The following cell establishes an `sqlalchemy` connection to trino

By default, `attach_trino_engine()` expects the following environment variables,
which should be supplied by the `credentials.env` file above:

```
TRINO_USER
TRINO_PASSWD
TRINO_HOST
TRINO_PORT
```

In [3]:
# connect to the Trino DB and return a sqlalchemy engine
engine = osc.attach_trino_engine()

## Obtain query results directly from sqlalchemy

The following cell shows an example of running a simple SQL query,
and obtaining the results directly from the `sqlalchemy` engine.

In [4]:
sql = """
select current_groups()
"""
qres = engine.execute(sql)
qres.fetchall()

[(['hive_ingest', 'aicoe_osc_demo', 'admins'],)]

## Load an SQL query into pandas
The `pandas` library can read a sql query directly into a DataFrame
using an `sqlalchemy` engine, as shown in the following cell.

Note the use of `convert_dtypes()` to tell pandas to assess the data types of the columns.

In [5]:
import pandas as pd

sql = """
show catalogs
"""

# execute the SQL query and load it into a pandas dataframe
df = pd.read_sql(sql, engine)

# assess the column data types from their contents:
df = df.convert_dtypes()

# display the dataframe
df

,Catalog
0,jmx
1,kafka_fx
2,osc_datacommons_dev
3,osc_datacommons_hive_ingest
4,riskthinking
5,system


## check the column data types

You can check the column types returned for your query using the `info` DataFrame method:

In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Catalog  6 non-null      string
dtypes: string(1)
memory usage: 176.0 bytes
